### Parameter Estimation using basico


In [ ]:
import os
import pandas as pd
from basico import *
 
 
model_path = "/home/wilson0003/Copasi/no_x.cps"
 
 
experiment_files = [
"/home/wilson0003/Copasi/text files/kinetics_with_intermediate_min.txt",
"/home/wilson0003/Copasi/text files/Kinetics.txt",
]
 
 
all_results = {}
 
 
for i, exp_file in enumerate(experiment_files):
exp_name = f"exp_{i+1}" # Unique name for each experiment
 
 
if not os.path.exists(exp_file):
print(f"⚠️ Skipping {exp_file} - File not found!")
continue
 
print(f"\n🔹 Using experiment file: {exp_file}")
 

model = load_model(model_path)
 

exp_data = pd.read_csv(exp_file, sep="\t") # Ensure correct separator
print(f"First rows of {exp_name} experimental data:\n", exp_data.head())
 

if exp_name in get_experiment_names():
remove_experiment(exp_name)
 

add_experiment(exp_name, exp_data)
print(f"✅ Experiment {exp_name} added successfully!")
 

mapping = get_experiment_mapping(exp_name)
mapping["cn"] = mapping["cn"].str.replace("ParticleNumber",
"Concentration")
 

print(f"🔹 Running Parameter Estimation for {exp_name}...")
run_parameter_estimation(method="Particle Swarm", update_model=False)
# 🔹 Prevents model contamination
 
fit_stats = get_fit_statistic(include_parameters=True)
 

parameters_with_sd = []
for param in fit_stats.get("parameters", []):
parameters_with_sd.append({
"name": param["name"],
"value": param["value"],
"std_dev": param.get("std_dev", "N/A"), # Get SD if available
"coeff_of_variation": param.get("coeff_of_variation", "N/A")
})
 

result_fit = pd.DataFrame(parameters_with_sd)
print(f"📊 Fitted Parameters for {exp_name}:\n", result_fit)
 

all_results[exp_name] = result_fit
 

plot_per_dependent_variable()
 
 
all_results_df = pd.concat(all_results, names=["Experiment"])
all_results_df.to_csv("/home/wilson0003/Copasi/parameter_estimation_results_with_sd.csv")
print("\n✅ All parameter estimations completed & saved!")

In [ ]:
I would remove all experiments using 'remove_experiments()' after the call
to load. Otherwise you'll leave the experiment which is named 'Experiment'
in and this will change your results a bit in the first place.
 
Also, you might want to consider using a local method (like LM:
`run_parameter_estimation(method=PE.LEVENBERG_MARQUARDT,
update_model=False) # 🔹 Prevents model contamination`). That is more
likely to run the along the same trajectory. Particle Swarm creates a swarm
randomly positioned in your parameter space, and since you are not
controlling the seed I would be surprised if you got precisely the same
results every time you run.
 
I hope this helps,
best
Frank